# Project Code 
Written by Brendan

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math
from IPython.display import display

%run RocketEquations.py

[3.4, 0.21, 0.26, 2959, 2428, 8.9, 386, 1.26]


## Body of Code

In [2]:
# Super Sonic Nozzle Plotter / Chamber Plotter (For Design Project)
DesignProject2 = widgets.Button(description="Design Project")
outputDP2 = widgets.Output()

def DP2(b):
    with outputDP2:
# Table Selection
        
        #AltTable=SIAltTable
        #TempTable=SITempTable
        #PressTable=SIPressTable
        
# Fuel Selection
        Prop=PropType(Oxidizer.value,Fuel.value)
        r=Prop[0]      # Mass prop ratio
        rV=Prop[1]     # Volume prop ratio
        SG=Prop[2]     # Average specific gravity
        t1=4380+460     # Combustion Temperature K
        #cst=Prop[4]    # C Star (speed of sound / thrust coeff)
        MW=Prop[5]     # Molar Mass kg/mol
        Isp=Prop[6]    #Specific Impulse
        k=Prop[7]      # Ratio of Specific Heats
        
# Pre-setting Data Point Arrays
        X=[]
        Y=[]
        X15=[]
        Y15=[]
        
# Constants
        g=32.2         # m/s^2
        R=1544         # kJ/mol-K
        VCF=0.98       # Velocity Correction
        TCF=0.97       # Thrust Correction
        
# Calculating Required Parameters and Tabulating
        F_IDEAL=10000  # N
        p1=1000     # Pa
        p2=1.58       # Pa
        p3=p2
        ToF=2.75*60     # seconds
                
        dwo=71.1       # Weight Density 
        dwf=4.4         # Weight Density 
        F_TRUE=F_IDEAL*TCF
        p2p1=p2/p1
        
        v2IDEAL=ExhaustVUS(k,R,t1,MW,p2p1)
        v2TRUE=v2IDEAL*VCF
        
        IspIDEAL=SpecificImpulseV(v2IDEAL,g)
        IspTRUE=SpecificImpulseV(v2TRUE,g)
        
        CfIDEAL=CfIdeal(k,p2p1)
        CfTRUE=CfIDEAL*TCF
        
        AtIDEAL=F_IDEAL/(CfIDEAL*p1)
        AtTRUE=F_IDEAL/(CfTRUE*p1)
        AR=AreaRatio(k,p2p1)
        A2IDEAL=AtIDEAL*AR
        A2TRUE=AtTRUE*AR
        
        M2=1
        Res=0.0001
        ARint=AR**-1
        
        while ARint<AR:
            M2=M2+Res
            ARint=ExpansionRatio(k,M2)
            
        wdtIDEAL=TotalWdot(F_IDEAL,IspIDEAL)
        wdtTRUE=TotalWdot(F_TRUE,IspTRUE)
        wdoIDEAL=OxWdot(wdtIDEAL,r)
        wdoTRUE=OxWdot(wdtTRUE,r)
        wdfIDEAL=FuelWdot(wdtIDEAL,r)
        wdfTRUE=FuelWdot(wdtTRUE,r)
        
        wpIDEAL=wdtIDEAL*ToF
        wpTRUE=wdtTRUE*ToF
        woIDEAL=wdoIDEAL*ToF
        woTRUE=wdoTRUE*ToF
        wfIDEAL=wdfIDEAL*ToF
        wfTRUE=wdfTRUE*ToF
        
        VdoIDEAL=wdoIDEAL/dwo
        VdoTRUE=wdoTRUE/dwo
        VdfIDEAL=wdfIDEAL/dwf
        VdfTRUE=wdfTRUE/dwf
        
        VoIDEAL=VdoIDEAL*ToF
        VoTRUE=VdoTRUE*ToF
        VfIDEAL=VdfIDEAL*ToF
        VfTRUE=VdfTRUE*ToF
            

        
# Calculating Nozzle Dimensions
        Rt=(AtTRUE/3.141592)**0.5 # in
        Re=Rt*(AR)**0.5           # in
        AngleN=35                 # Degrees
        AngleE=5
        L=0.8*((Re-Rt)/math.tan(math.radians(15)))
        
# Converging Nozzle
        for i in range (-135,-90,1):
            X.append(1.5*Rt*math.cos(math.radians(i)))
            Y.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)
            X15.append(1.5*Rt*math.cos(math.radians(i)))
            Y15.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)

# Throat and Circular Divergence
        for i in range (-90,(AngleN-90),1):
            X.append(0.382*Rt*math.cos(math.radians(i)))
            Y.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

        for i in range (-90,(15-90),1):    
            X15.append(0.382*Rt*math.cos(math.radians(i)))
            Y15.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

# Bell Divergence
        Ex=L
        Ey=Re
        Nx=0.382*Rt*math.cos(math.radians(AngleN-90))
        Ny=0.382*Rt*math.sin(math.radians(AngleN-90))+0.382*Rt+Rt
        m1=math.tan(math.radians(AngleN))
        m2=math.tan(math.radians(AngleE))
        C1=Ny-m1*Nx
        C2=Ey-m2*Ex
        Qx=(C2-C1)/(m1-m2)
        Qy=(m1*C2-m2*C1)/(m1-m2)

        for t in range (0,100,1):
            t=t/100
            X.append(((1-t)**0.5)*Nx+2*(1-t)*t*Qx+(t**2)*Ex)
            Y.append(((1-t)**0.5)*Ny+2*(1-t)*t*Qy+(t**2)*Ey)

            
# Combustion Chamber Work
        Dt=Rt*2
        Dc=Dt*2
        Rc=Dt
        Ac=0.5*math.pi*Rc**2
        Beta=42
        
        Lst=31.2 #m
        Vchamber=Lst*AtTRUE
        Lconv=-Rc/math.tan(math.radians(Beta))
        Vconv=(1/3)*math.pi*(Rc**2+Rc*Rt+Rt**2)*Lconv
        
        Vcyl=Vchamber-Vconv
        Lcyl=Vcyl/Ac
        
        Lchamber=Lcyl+Lconv
        
        CDXinit=-Lchamber
        CDX=[CDXinit]
        CDY=[Rc]
        CDXcrv=X[0]-(Rc-Y[0])
        
        while CDXinit<CDXcrv:
            CDXinit=CDXinit+0.01
            CDY.append(Dt)
            CDX.append(CDXinit)
            
        CDYinit=Rc
        
        while CDXinit<X[0]:
            CDXinit=CDXinit+0.01
            CDYinit=CDYinit-0.01
            CDY.append(CDYinit)
            CDX.append(CDXinit)
            
# Injector Work
        mdo=wdoTRUE
        mdf=wdfTRUE
        
        dp=0.2*p1
        Cd=0.9
        yo=-10     #Oxy injection angle
        yf=30      #fuel injection angle
        Ao=(mdo/(Cd*(2*(dwo)*dp)**0.5))
        Af=(mdf/(Cd*(2*(dwf)*dp)**0.5))
        print(Ao,Ac)
        
        OHole=0.01968504**2*0.25*math.pi
        Num=Ao//OHole
        if Num%2>0:
            Num=Num+1
        else:
            Num=Num
        FHole=Af/Num
        
        CenterR=((6*0.01968504)*12)/(2*math.pi)
        Spacing=(Rc-CenterR)/(Num/12)
        
        
        print(Num,'  ',CenterR,'  ',
              (FHole/math.pi)**0.5,'  ',
              (OHole/math.pi)**0.5,'  ',
              Spacing,'  ',Rc)
               
        
        
        
        
        
# Plotting the Bell Nozzle 
        NegY=[i*-1 for i in Y]
        NegCDY=[i*-1 for i in CDY]
        plt.xlabel('Length in Meters')
        plt.ylabel('Width in Meters')
        plt.plot(X,Y,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X,NegY,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-20,30])
        plt.ylim([-16.2,16.2])
        plt.title('Rao Nozzle Plot')
        plt.show()

# 15 Degree Half Angle Divergence
        Nx15=0.382*Rt*math.cos(math.radians(15-90))
        Ny15=0.382*Rt*math.sin(math.radians(15-90))+0.382*Rt+Rt

        xtemp=int(round(Nx))
        while math.tan(math.radians(15))*xtemp+Ny15 < Re:
            X15.append(xtemp)
            Y15.append(math.tan(math.radians(15))*xtemp+Ny15)
            xtemp=xtemp+0.01

# Plotting 15 degree Half Angle Nozzle
        NegY15=[i*-1 for i in Y15]
        plt.xlabel('Length in Inches')
        plt.ylabel('Width in Inches')
        plt.plot(X15,Y15,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X15,NegY15,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-20,30])
        plt.ylim([-16.2,16.2])
        plt.title('15 Degree Angle Nozzle Plot')
        plt.show()
        
# Combustion Instability
        RadialL=2*(Dc)/12
        TanL=2*math.pi*(Dc)/12
        AxialL=2*(Lchamber)/12
        
        a=(k*t1*R/MW)**0.5
        
        RadialF=a/RadialL
        TanF=a/TanL
        AxialF=a/AxialL
        
        print('---------------------------------------------------------')
        print('---------Velocities, Areas, Flowrates, Weights-----------')
        print('---------------------------------------------------------\n')
        
        print('Value                      Ideal         Actual')
        print('Exit Velocity:             ','{0:.0f}'.format(v2IDEAL),'(m/s)    '
              '{0:.0f}'.format(v2TRUE),'(m/s)\n')
        print('Specific Impulse:             ','{0:.0f}'.format(IspIDEAL),'(m/s)    '
              '{0:.0f}'.format(IspTRUE),'(m/s)\n')
        print('Thrust Coeff:             ','{0:.2f}'.format(CfIDEAL),'(m/s)    '
              '{0:.2f}'.format(CfTRUE),'(m/s)\n')
        print('Throat Area:               ','{0:.4f}'.format(AtIDEAL),'(m^2)      '
              '{0:.4f}'.format(AtTRUE),'(m^2)\n')
        print('Exit Area:                 ','{0:.3f}'.format(A2IDEAL),'(m^2)    '
              '{0:.3f}'.format(A2TRUE),'(m^2)\n')
        print('Exit Mach:                 ','{0:.2f}'.format(M2),'(~)        '
              '{0:.2f}'.format(M2),'(~)\n')
        print('Total Weight Flow Rate:    ','{0:.2f}'.format(wdtIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdtTRUE),'(N/s)\n')
        print('Oxidizer Weight Flow Rate: ','{0:.2f}'.format(wdoIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdoTRUE),'(N/s)\n')
        print('Fuel Weight Flow Rate:     ','{0:.2f}'.format(wdfIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdfTRUE),'(N/s)\n')
        print('Propellant Weight:         ','{0:.0f}'.format(wpIDEAL),'(N)    '
              '{0:.0f}'.format(wpTRUE),'(N)\n')
        print('Oxidizer Weight:           ','{0:.0f}'.format(woIDEAL),'(N)     '
              '{0:.0f}'.format(woTRUE),'(N)\n')
        print('Fuel Weight:               ','{0:.0f}'.format(wfIDEAL),'(N)     '
              '{0:.0f}'.format(wfTRUE),'(N)\n')
        print('Oxidizer Volume Flow Rate: ','{0:.4f}'.format(VdoIDEAL),'(m^3/s)    '
              '{0:.4f}'.format(VdoTRUE),'(m^3/s)\n')
        print('Fuel Volume Flow Rate:     ','{0:.4f}'.format(VdfIDEAL),'(m^3/s)    '
              '{0:.4f}'.format(VdfTRUE),'(m^3/s)\n')
        print('Oxidizer Volume:           ','{0:.4f}'.format(VoIDEAL),'(m^3)    '
              '{0:.4f}'.format(VoTRUE),'(m^3)\n')
        print('Fuel Volume:               ','{0:.4f}'.format(VfIDEAL),'(m^3)    '
              '{0:.4f}'.format(VfTRUE),'(m^3)\n')
        
        print('---------------------------------------------------------')
        print('-------------------------Frequencies---------------------')
        print('---------------------------------------------------------\n')
        print('Nozzle Length:','{0:.3f}'.format(L),'(m)\n')
        
        print('L* (Assumed):',Lst,'(m)')
        print('Convergence Angle (Assumed): 42 (deg)')
        print('Converging Duct Length (Calculated):','{0:.3f}'.format(-CDXcrv),'(m)')
        print('Cylindrical Duct Length (Calculated):',Lcyl,'(m)')
        print('Chamber Diameter (Calculated):','{0:.3f}'.format(Dc),'(m)\n')
        
        print('Radial Resonance Frequency:','{0:.2f}'.format(RadialF),'(Hz)')
        print('Tangential Resonance Frequency:','{0:.2f}'.format(TanF),'(Hz)')
        print('Axial Resonance Frequency:','{0:.2f}'.format(AxialF),'(Hz)\n')
        
        print('---------------------------------------------------------')
        print('-------------------------Injector------------------------')
        print('---------------------------------------------------------\n')
        
        print('Oxidizer Hole Diameter:','{0:.5f}'.format(2*(FHole/math.pi)**0.5),'in')
        print('Fuel Hole Diameter:','{0:.5f}'.format(2*(OHole/math.pi)**0.5),'in')
        print('Number of Pairs:', Num)
        print('Number of Branges of Pairs: 12')
        
DesignProject2.on_click(DP2)

In [3]:
Oxidizer=widgets.Dropdown(
    options=['Oxygen','Flourine','Nitrogen Tetroxide','Nitric Acid','Hydrogen Peroxide'],
    value='Oxygen',
    description='Oxidizer:',
    disabled=False,
)

Fuel=widgets.Dropdown(
    options=['Methane','Hydrazine','Hydrogen','RP-1','UDMH','UDMH/Hydrazine','MMH'],
    value='Hydrogen',
    description='Fuel:',
    disabled=False,
)

DispWindows1=widgets.HBox([Oxidizer,Fuel],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

display(DispWindows1)
display(DesignProject2, outputDP2)

Button(description='Design Project', style=ButtonStyle())

Output()